### Part 3 -> Multiturn conversation dataset generation

In [1]:
import json
from tqdm import tqdm
from openai import OpenAI
import random
import pandas as pd

path_to_api_key: str = "../API_KEY.txt"
my_api_key = open(path_to_api_key, 'r').read()
client = OpenAI(api_key=my_api_key)
gpt_model: str = "gpt-4o"

In [ ]:
# Multiturn Class that represents one interaction mode (system prompt for Meditron and User, both simulated by gpt-4o)

class multiturn_style:
    def __init__(self, id, country, setting = "high", nbr_of_turns = 1, mode = "normal",
                 cb_style = "flat text", cb_length = "short", cb_scientific = "standard",
                 u_length = "short",  u_scientific = False, u_specialty = "physician", u_bad_grammar = False):
         
        # Initialize situational attributes
        self.id = id # based on prompt origin and prompt number
        self.country = country # country the user is working in
        self.setting = setting  # Can be 'high', 'low', or 'war' -> not used so far
        self.number_of_turns = nbr_of_turns # Number of turns (1 = 1 question + 1 answer)
        self.mode = mode # Three different modes: "chat", "normal", "emergency"

        # Initizalize attributes that determine system prompt of Meditron
        self.chatbot_length = cb_length # Length of chatbots answer -> possibilities: short, long, exactly_2_sentences, exactly_4_sentences
        self.chatbot_style = cb_style # Style of chatbots answer -> possibilities: flat text, bullets, step by step
        self.chatbot_scientific = cb_scientific # If chatbot uses a lot of scientific term -> possibilities: popularization, standard, scientific

        # Initialize attributes that determine system prompt of user
        self.user_length = u_length # Length of user questions -> possibilities: short, long, exactly_1_sentence, exactly_3_sentences
        self.user_scientific = u_scientific # User uses a lot of very scientific terms -> not used so far
        self.user_specialty = u_specialty
        self.bad_grammar = u_bad_grammar # user has bad grammar / does a lot of spelling mistakes -> not used so far

        # Fixed attributes
        self.base_system_prompt_chatbot = f'''You are a medical AI chatbot designed to assist health care workers working in {self.country} by answering their questions. \nThe style of your answers must follow these rules: \n'''
        self.base_system_prompt_user = f'''You are a {self.user_specialty} working in {self.country} using a medical AI chatbot to help you taking care of your patients. \nThe initial question that you ask the medical AI chatbot has already been provided. You are now interacting with the medical AI chatbot and asking follow up questions, based on the answers provided by the medical AI chatbot. \nThe style of the follow up question you ask must follow these rules: \n'''

        self.number_of_turns_done = 0

    # methods

    # adds sytle specification to the system prompt of meditron
    def system_prompt_chatbot(self) -> str:
        # Chatbot can be used in chat, normal or emergency mode
        match self.mode:
            case "chat":
                self.base_system_prompt_chatbot += "The user is using you (the medical AI chatbot) in chat mode. Provide short and concise answers that make it possible for the user to chat with you. Ask the user follow up questions about his question if his question is not clear enough. \n"
            case "emergency":
                self.base_system_prompt_chatbot += "The user is using you (the medical AI chatbot) during a medical emergency. Provide short, concises and well structured answers. Your answers have to be clear and very easy to understand quickly in a high presure and stres environment \n"
            case "normal":
                # Set epected length of chatbots answer
                match self.chatbot_length:
                    case "short":
                        self.base_system_prompt_chatbot += "Provide short and concises answers. \n"
                    case "long":
                        self.base_system_prompt_chatbot += "Provide long and detailed answers. \n"
                    case "exactly_2_sentences":
                        self.base_system_prompt_chatbot += "All your answers should be exactly 2 sentences long. \n"
                    case "exactly_4_sentences":
                        self.base_system_prompt_chatbot += "All your answers should be exactly 4 sentences long. \n"
                    case _:
                        raise ValueError(f"Invalid chatbot_length: '{self.chatbot_length}'")
                
                # Set expected style of chatbots answer
                match self.chatbot_style:
                    case "flat text":
                        self.base_system_prompt_chatbot += "Provide your answers in flat text. \n"
                    case "bullets":
                        self.base_system_prompt_chatbot += "Provide your answers using bullet points. \n"
                    case "step by step":
                        self.base_system_prompt_chatbot += "Provide your answers by giving the physician step by step advice on how to solve the question they asked. \n"
                    case _:
                        raise ValueError(f"Invalid chatbot_style: '{self.chatbot_style}'")
            
        # Add if the style of the answers should be very scientific
        match self.chatbot_scientific:
            case "popularization":
                self.base_system_prompt_chatbot += "Provide your answers using easy to understand terms and avoid using scientific terms that are difficult to understand. \n"
            case "standard":
                self.base_system_prompt_chatbot += "" # Don't add anything if standard
            case "scientific":
                self.base_system_prompt_chatbot += "Provide your answers using a lot of very specific scientific terms. \n"
            case _:
                raise ValueError(f"Invalid chatbot_scientific: '{self.chatbot_scientific}'")

        return self.base_system_prompt_chatbot
    
    def system_prompt_user(self) -> str:

        # User behavior is characterized by 3 different modes: chat, normal or emergency mode
        match self.mode:
            case "chat":
                self.base_system_prompt_user += f"You (the {self.user_specialty} using the medical AI chatbot) are using the medical AI chatbot in chat mode. Ask short and concise questions that make it possible for the chatbot to chat with you. Ask follow up questions about the answers from the chatbot if the answers are not clear enough. \n"
            case "emergency":
                self.base_system_prompt_chatbot += f"You (the {self.user_specialty} using the medical AI chatbot) are using the medical AI chatbot during a medical emergency. Ask short and fastly written questions. \n"
            case "normal":
                # Set epected length of chatbots answer
                match self.user_length:
                    case "short":
                        self.base_system_prompt_user += "Ask short and concises questions. \n"
                    case "long":
                        self.base_system_prompt_user += "Ask long and detailed questions. \n"
                    case "exactly_1_sentence":
                        self.base_system_prompt_user += "All your questions should be exactly 1 sentence long. \n"
                    case "exactly_3_sentences":
                        self.base_system_prompt_user += "All your questions should be exactly 3 sentences long. \n"
                    case _:
                        raise ValueError(f"Invalid user_length: '{self.user_length}'")
                
            
        # Add if the style of the answers should be very scientific
        match self.user_scientific:
            case "popularization":
                self.base_system_prompt_chatbot += "Ask follow up questions using easy to understand terms and avoid using scientific terms that are difficult. \n"
            case "standard":
                self.base_system_prompt_chatbot += "" # Don't add anything if standard
            case "scientific":
                self.base_system_prompt_chatbot += "Ask follow up questions using a lot of very specific scientific terms. \n"
            case _:
                raise ValueError(f"Invalid user_scientific: '{self.user_scientific}'")
                                        
        return self.base_system_prompt_user


In [24]:
# Helper function get sample

def get_sample(choices, weights):
    return random.choices(choices, weights, k=1)[0]

# Function that creates the multiturn_style objects, following the distributions specified

countries = pd.read_csv("../resources/countries_by_income_category.csv")
sampled_country = random.choice(countries.iloc[:, 0].tolist())
id = "0-122"

# Tasks
# 0 - "Diagnosing Symptoms", 1 - "Predictive Analytics", 2 - "Drug Interaction Checks", 3 - "Treatment Recommendations"
# 4 - "Medication Management", 5 - "Patient Education", 6 - "Clinical Documentation", 7 - "Lab Test Interpretation"
# 8 - "Health Risk Assessment", 9 - "Nutrition and Diet Planning", 10 - "Language Translation", 11 - "Emergency Triage"
# 12 - "Health Policy Guidance", 20 - Specialty times domain prompts

def get_multiturn_style(id, sampled_country, profession = "no", specialty = "no", domain = "no"): # id is task_id - specialty_id

    # Usage mode: chat, normal, emergency (depening on id -> since )
    mode_choices = ["chat", "normal", "emergency"]
    match int(id.split("-")[0]):
        case n if n in [1, 4, 5, 8, 9, 10, 12]: # Non Emergency cases
            mode = get_sample(choices=mode_choices, weights=[0.4, 0.6, 0])
        case 6: # Non Emergency and non chat option
              mode = "normal"
        case n if n in [0, 2, 3, 7]: # All cases possible
            mode = get_sample(choices=mode_choices, weights=[0.4, 0.5, 0.1])
        case 11: # Very good for emergency
            mode = get_sample(choices=mode_choices, weights=[0.3, 0.3, 0.4])
        case 20:
            if specialty == "Emergency Medicine" or domain == "Emergency Medicine":
                mode = "emergency"
            else:
                mode = get_sample(choices=mode_choices, weights=[0.4, 0.6, 0])
        case _:
            raise ValueError(f"Invalid id: '{id}'")     
    
    # Number of turn
    nbr_of_turns_choices = [1, 2, 3]
    match mode:
        case s if s in ["chat", "emergency"]:
            nbr_of_turns = get_sample(choices=nbr_of_turns_choices, weights=[0.2, 0.3, 0.5])
        case "normal":
            nbr_of_turns = get_sample(choices=nbr_of_turns_choices, weights=[0.4, 0.3, 0.3])

    # Length of chatbots answer -> possibilities: short, long, exactly_2_sentences, exactly_4_sentences
    chatbot_length_choices = ["short", "long", "exactly_2_sentences", "exactly_4_sentences"]
    cb_length = get_sample(choices=chatbot_length_choices, weights=[0.3, 0.5, 0.1, 0.1])

    # Style of chatbot's answer -> possibilities: chat, flat text, bullets, step by step
    chatbot_style_choices = ["flat text", "bullets", "step by step"]
    cb_style = get_sample(choices=chatbot_style_choices, weights=[0.5, 0.25, 0.25])

    # If chatbot uses a lot of scientific term -> possibilities: popularization, standard, scientific
    chatbot_scientific_choices = ["popularization", "standard", "scientific"]
    cb_scientific = get_sample(choices=chatbot_scientific_choices, weights=[0.1, 0.8, 0.1])

    # Length of user questions -> possibilities: chat, short, long, exactly_1_sentence, exactly_3_sentences
    user_length_choices = ["short", "long", "exactly_1_sentence", "exactly_3_sentences"]
    u_length = get_sample(choices=user_length_choices, weights=[0.4, 0.3, 0.2, 0.1])

    # If the user uses a lot of scientific term -> possibilities: popularization, standard, scientific
    user_scientific_choices = ["popularization", "standard", "scientific"]
    u_scientific = get_sample(choices=user_scientific_choices, weights=[0.1, 0.8, 0.1])

    if profession == "no": # task_x_subtopics case
        return multiturn_style(id=id, country=sampled_country, nbr_of_turns=nbr_of_turns, mode=mode,
                            cb_length=cb_length, cb_style=cb_style, cb_scientific=cb_scientific,
                            u_length=u_length, u_scientific=u_scientific)
    else:
        return multiturn_style(id=id, country=sampled_country, nbr_of_turns=nbr_of_turns, mode=mode,
                            cb_length=cb_length, cb_style=cb_style, cb_scientific=cb_scientific,
                            u_length=u_length, u_scientific=u_scientific, u_specialty= profession + "specialized in " + specialty)


multiturn_style_1 = get_multiturn_style(id, sampled_country)

print(multiturn_style_1.system_prompt_chatbot())
print(multiturn_style_1.system_prompt_user())

You are a medical AI chatbot designed to assist physicians working in Croatia by answering their questions. 
The style of your answers must follow these rules: 
The user is using you (the medical AI chatbot) in chat mode. Provide short and concise answers that make it possible for the user to chat with you. Ask the user follow up questions about his question if his question is not clear enough. 

You are a physician working in Croatia using a medical AI chatbot to help you taking care of your patients. 
The initial question that you ask the medical AI chatbot has already been provided. You are now interacting with the medical AI chatbot and asking follow up questions, based on the answers provided by the medical AI chatbot. 
The style of the follow up question you ask must follow these rules: 
You (the physician using the medical AI chatbot) are using the medical AI chatbot in chat mode. Ask short and concise questions that make it possible for the chatbot to chat with you. Ask follow 

In [25]:
for i in range(10000):
    test = get_multiturn_style(str(i%12)+"-112", "Switzerland")

print(test.system_prompt_chatbot())
print(test.system_prompt_user())

You are a medical AI chatbot designed to assist physicians working in Switzerland by answering their questions. 
The style of your answers must follow these rules: 
The user is using you (the medical AI chatbot) in chat mode. Provide short and concise answers that make it possible for the user to chat with you. Ask the user follow up questions about his question if his question is not clear enough. 

You are a physician working in Switzerland using a medical AI chatbot to help you taking care of your patients. 
The initial question that you ask the medical AI chatbot has already been provided. You are now interacting with the medical AI chatbot and asking follow up questions, based on the answers provided by the medical AI chatbot. 
The style of the follow up question you ask must follow these rules: 
You (the physician using the medical AI chatbot) are using the medical AI chatbot in chat mode. Ask short and concise questions that make it possible for the chatbot to chat with you. Ask

In [ ]:
# Test mode
# Generate multiturn conversations between Meditron (simulated by gpt-4o) and a Physician (simulated by gpt-4o)

# System prompts
content_system = "You are a medical AI chatbot that answers questions from physicians."
content_user = "You are a physician that interacts with a medical AI chatbot. You see the chat history between you (user) and the medical AI chatbot (chatbot). Ask a follow up question or write 'I AM ALL GOOD THANKS', if there is no follow up question to ask."

# store medical AI tasks in a list
path_to_prompts = "../results/parsed_prompts_tasks_x_subtopics.json" 
with open(path_to_prompts, 'r') as file:
    task_x_subtopics_prompts = json.load(file)  # Load JSON data from file
initial_physician_prompt = task_x_subtopics_prompts[0]["prompt"]
print(initial_physician_prompt)

# Function that generates the next step of a conversation

def next_conversation_step(system_prompt, chat_histroy, verbose = False):
    if system_prompt == "user":
        content = "You are a physician that interacts with a medical AI chatbot. You see the chat history between you (user) and the medical AI chatbot (chatbot). Ask a follow up question or write 'I AM ALL GOOD THANKS', if there is no follow up question to ask."
    else:
        content = "You are a medical AI chatbot that answers questions from physicians."

    prompt = chat_histroy

    if verbose:
        print("Send prompt to GPT:")
        print("#########")
        print(prompt)
        print("########")

    completion = client.chat.completions###.create( # remove ### (security stop)
        model= gpt_model,
        messages=[
            {"role": "system", "content": content},
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ]
    )
    response = completion.choices[0].message.content
    
    if verbose:
        print("Receiving responses from GPT...")
        print(response)
    return chat_histroy + "\n" + f"({system_prompt}) " + response

conversation_history = []
conversation_history.append("(user) " + initial_physician_prompt)
for i in tqdm(range(0)): ### Change range -> will cost money
    if i % 2 == 0:
        if conversation_history[-1] != "I AM ALL GOOD THANKS":
            conversation_history.append(next_conversation_step("chatbot", conversation_history[-1]))
    else:
        conversation_history.append(next_conversation_step("user", conversation_history[-1]))


In [ ]:
# Bastien Code for inspiration

def system_prompt(self, final=False) -> str:

        # First sentence options
        # Updated first sentence options
        if self.brief:
            first_sentence_options = [
                "As a doctor in {}, you are in {} Your task is to diagnose a patient by asking concise questions and providing brief reasoning. Keep your explanations short and to the point, and ask pertinent questions.",
                "You are practicing medicine in {}. {} Your mission is to determine the patient's diagnosis through concise inquiry, being efficient in your thought process. Make your thinking clear but succinct, then pose relevant questions.",
                "While working as a physician in {}, {} Your role is to identify the patient's ailment by asking concise questions and efficiently analyzing your thoughts. Be explicit yet brief in your reasoning, and ask appropriate questions.",
                "In {}, you find yourself in {} Your task is to diagnose a patient by asking concise questions and providing brief reasoning. Keep your explanations short and to the point, and ask pertinent questions.",
                "Serving as a doctor in {}, {} Your mission is to determine the patient's diagnosis through concise inquiry, being efficient in your thought process. Make your thinking clear but succinct, then pose relevant questions.",
                "As a physician in {}, you are confronted with {} Your role is to identify the patient's ailment by asking concise questions and efficiently analyzing your thoughts. Be explicit yet brief in your reasoning, and ask appropriate questions.",
            ]
        elif self.bullet:
            first_sentence_options = [
                "As a doctor in {}, you are in {} Your task is to diagnose a patient by asking concise questions and providing brief reasoning. Be explicit and to the point, present your reasoning and questions in bullet points.",
                "You are practicing medicine in {}. {} Your mission is to determine the patient's diagnosis through concise inquiry, being efficient in your thought process. Make your thinking clear and format your reasoning and questions as bullet points.",
                "While working as a physician in {}, {} Your role is to identify the patient's ailment by asking concise questions and efficiently analyzing your thoughts. Be explicit and present your questions and reasoning in bullet points.",
                "In {}, you find yourself in {} Your task is to diagnose a patient by asking concise questions and providing brief reasoning. Make your thinking clear and use bullet points to present your reasoning and questions.",
                "Serving as a doctor in {}, {} Your mission is to determine the patient's diagnosis through concise inquiry, being efficient in your thought process. Make your thinking clear and present your reasoning and questions in bullet points.",
                "As a physician in {}, you are confronted with {} Your role is to identify the patient's ailment by asking concise questions and efficiently analyzing your thoughts. Be explicit and format your reasoning and questions as bullet points.",
            ]
        else:
            first_sentence_options = [
                "As a doctor in {}, you are in {} Your task is to diagnose a patient by asking questions and carefully considering your thoughts. Explain your reasoning and ask pertinent questions.",
                "You are practicing medicine in {}. {} Your mission is to determine the patient's diagnosis through inquiry, being thorough in your thought process. Make your thinking explicit, then pose relevant questions.",
                "While working as a physician in {}, {} Your role is to identify the patient's ailment by asking questions and thoroughly analyzing your thoughts. Be explicit about your reasoning, and ask appropriate questions.",
                "In {}, you find yourself in {} Your task is to diagnose a patient by asking questions and carefully considering your thoughts. Explain your reasoning and ask pertinent questions.",
                "Serving as a doctor in {}, {} Your mission is to determine the patient's diagnosis through inquiry, being thorough in your thought process. Make your thinking explicit, then pose relevant questions.",
                "As a physician in {}, you are confronted with {} Your role is to identify the patient's ailment by asking questions and thoroughly analyzing your thoughts. Be explicit about your reasoning, and ask appropriate questions.",
            ]

        first_sentence = random.choice(first_sentence_options).format(self.country, random.choice(setting_doctor[self.setting]))

        # Test request sentence for high resource setting
        if self.setting == "high" and self.infs < self.MAX_INFS:
            test_request_options = [
                "You may also request tests if they are available and can assist in your diagnosis, using the format \"REQUEST TEST: [test]\". For instance, \"REQUEST TEST: Chest_X-Ray\".",
                "If helpful and accessible, you should request tests to aid your diagnosis using the format \"REQUEST TEST: [test]\". Example: \"REQUEST TEST: Chest_X-Ray\".",
                "Feel free to request any available tests that might help in diagnosing, using \"REQUEST TEST: [test]\". For example, \"REQUEST TEST: Chest_X-Ray\".",
            ]
            test_request_sentence = random.choice(test_request_options)
        else:
            test_request_sentence = ""

        # Question limit sentence options
        if final:
            question_limit_options = [
                "You are only allowed to ask {} questions in total before you must make a decision.",
                "You have a limit of {} questions to ask before making a diagnosis.",
                "You can ask up to {} questions before you need to decide.",
                "You may ask a maximum of {} questions before diagnosing.",
                "A total of {} questions are allowed before you must diagnose.",
                "You have {} questions in total to reach your diagnosis.",
                "Only {} questions can be asked before making your decision.",
                "You are permitted to ask up to {} questions before you must decide.",
            ]
            question_limit_sentence = random.choice(question_limit_options).format(self.MAX_INFS)
        else:
            question_limit_options = [
                "You are only allowed to ask {} questions in total before you must make a decision. You have asked {} questions so far.",
                "You have a limit of {} questions to ask before making a diagnosis. So far, you've asked {} questions.",
                "You can ask up to {} questions before you need to decide. Currently, you have asked {} questions.",
                "You may ask a maximum of {} questions before diagnosing. You have already asked {} questions.",
                "A total of {} questions are allowed before you must diagnose. Up to now, you've asked {} questions.",
                "You have {} questions in total to reach your diagnosis. So far, you've used {} questions.",
                "Only {} questions can be asked before making your decision. You have asked {} questions till now.",
                "You are permitted to ask up to {} questions before you must decide. At present, you have asked {} questions.",
            ]
            question_limit_sentence = random.choice(question_limit_options).format(self.MAX_INFS, self.infs)
        

        # Diagnosis instruction options
        diagnosis_instruction_options = [
            "Once you have decided to make a diagnosis, please type \"DIAGNOSIS READY: [diagnosis here]\".",
            "When ready to diagnose, enter \"DIAGNOSIS READY: [diagnosis here]\".",
            "After concluding your diagnosis, submit it using \"DIAGNOSIS READY: [diagnosis here]\".",
            "Upon reaching a diagnosis, please type \"DIAGNOSIS READY: [diagnosis here]\".",
            "Once you are confident in your diagnosis, enter \"DIAGNOSIS READY: [diagnosis here]\".",
            "When you have determined the diagnosis, submit it using \"DIAGNOSIS READY: [diagnosis here]\".",
            "After you have made a diagnosis, indicate it by typing \"DIAGNOSIS READY: [diagnosis here]\".",
            "When you are prepared to diagnose, please provide it using \"DIAGNOSIS READY: [diagnosis here]\".",
        ]
        diagnosis_instruction_sentence = random.choice(diagnosis_instruction_options)

        # Knows diagnosis sentence
        if self.knows_diagnosis and not final:
            knows_diagnosis_options = [
                " You suspect that the patient suffers from {}. This affects the questions you ask the patient.",
                " You have a hunch that the patient may have {}. This should influence your questioning.",
                " You believe the patient might be suffering from {}. Let this guide your questions.",
            ]
            knows_diagnosis_sentence = random.choice(knows_diagnosis_options).format(self.correct_diagnosis)
        else:
            knows_diagnosis_sentence = ""